In [114]:
import json
import pinecone
import tqdm
from langchain.vectorstores import Pinecone
from langchain.docstore.document import Document
from multiprocessing import Pool
import pandas as pd
import itertools


In [168]:
link = pd.read_csv('/Users/mandytoh/Downloads/mandy_dict.csv')
docs = pd.read_parquet('/Users/mandytoh/docs_index_dets.parquet')
merged = pd.merge(link, docs, on='docname')
json_merged = {item['mandy_docs']: item for item in merged.to_dict('records')}
filepath = '/Users/mandytoh/Projects/lux_google/material_science_emb.jsonl'
directory = '/Users/mandytoh/Projects/lux_google/material_science_emb/'


In [52]:
pinecone.init(api_key=API_KEY, environment=PINECONE_ENV)

In [122]:
pinecone.list_indexes()
index = pinecone.Index("paperqa-index")

In [201]:
#run this every now and then, check that the uploads # == upsert log #
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.1,
 'namespaces': {'docs': {'vector_count': 100},
                'texts': {'vector_count': 19720}},
 'total_vector_count': 19820}

In [165]:
# For any mistakes, ONLY RUN IF YOU KNOW NOTHING ELSE IS IN THERE
# index.delete(deleteAll='true', namespace='texts')

{}

In [207]:
def chunks(mini_file, batch_size, i):
    input_dict = []
    chunk_ids = []
    with open(mini_file, 'r') as file:
        for line in file:
            record = json.loads(line)
            if 'data' not in record[1]:
                # skip everything that errored out with openai api
                continue
            emb = record[1]['data'][0]['embedding']
            mandy_id = float(record[0]['doc'])
            metadata = json_merged[mandy_id]
            metadata['text'] = record[0]['input'].encode('utf-8', 'ignore').decode('utf-8', 'ignore')

            if 'mandy_docs' in metadata:
                metadata.pop('mandy_docs')
            #chunk_id in json can't be used, there are repeated elements, using random instead
            line_id = str(uuid.uuid4())
            chunk_ids.append(line_id)
            entry_c = (line_id, emb, metadata)
            input_dict.append(entry_c)
        chunk_log = f'/Users/mandytoh/Projects/lux_google/logs/{i}_ids.txt'
        with open(chunk_log, 'w') as file:
            for item in chunk_ids:
                file.write(f'{item}\n')
    it = iter(input_dict)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))
        
#mini_file = '/Users/mandytoh/Projects/lux_google/material_science_emb/output_1.jsonl'
# for filename in os.listdir(directory):
#     mini_file = os.path.join(directory, filename)
#     print('currently uploading file: ', mini_file)
i = 6
mini_file = f'/Users/mandytoh/Projects/lux_google/material_science_emb/output_{i}.jsonl'
print('currently uploading file: ', mini_file)
for ids_vectors_chunk in chunks(mini_file, batch_size=100, i = i):
    upsert_response = index.upsert(vectors=ids_vectors_chunk, namespace='texts')
    print(upsert_response)

currently uploading file:  /Users/mandytoh/Projects/lux_google/material_science_emb/output_6.jsonl
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_count': 100}
{'upserted_co

In [ ]:
# JSON exploratory breakdown, all the important fields
#k[0][1].keys() = ['object', 'data', 'model', 'usage']
#k[0][1]['data'][0].keys() = ['object', 'index', 'embedding']
#k[0][1]['data'][0]['embedding'] = list of floats
#k[0][0]['input'] = texts
#k[0][0].keys() = dict_keys(['model', 'input', 'doc', 'chunk_id'])

In [159]:
index.query(
  vector=emb,
  top_k=3,
  include_values=True,
    namespace='texts'
)

{'matches': [{'id': '0704.0411 | pg 1 | chunk 0',
              'score': 3.69548798e-06,
              'values': [-0.0173509791,
                         0.0232427455,
                         0.00557420496,
                         0.000964506355,
                         0.00915520918,
                         0.00667553255,
                         -0.0308912303,
                         -0.0211211685,
                         -0.0269994214,
                         -0.0324857906,
                         0.00492894836,
                         0.0205536131,
                         -0.0202563219,
                         0.00969573855,
                         0.0154456152,
                         0.00114440115,
                         0.0111484099,
                         0.00195772829,
                         -0.00177361059,
                         -0.00501340628,
                         -0.0141753722,
                         0.0082836058,
                         -0.01006

In [88]:
import os
#If you want to batch json files
def split_jsonl(input_path, output_dir, num_files):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    with open(input_path, 'r') as input_file:
        lines = input_file.readlines()
    
    num_lines = len(lines)
    lines_per_file = num_lines // num_files

    for i in range(num_files):
        start = i * lines_per_file
        end = num_lines if i == num_files - 1 else start + lines_per_file

        with open(os.path.join(output_dir, f'output_{i}.jsonl'), 'w') as output_file:
            output_file.writelines(lines[start:end])

# Usage
split_jsonl(filepath, '/Users/mandytoh/Projects/lux_google/material_science_emb/', 100)